In [1]:
import os
import json
import nibabel as nib
from nilearn.image import math_img
import numpy as np
from nibabel.processing import resample_from_to

In [2]:
# Load config file with path to raw, derived and output data
config_file = os.path.join(os.path.abspath(''), os.pardir, 'scripts', 'config.json')
with open(config_file) as json_data:
    config = json.load(json_data)

out_dir = config['output_dir']

In [3]:
res_dir = os.path.join(out_dir, 'results')
if not os.path.isdir(res_dir):
    os.mkdir(res_dir)

In [4]:
fsl_dir = os.environ['FSLDIR']
cort_atlas_file = os.path.join(fsl_dir, 'data', 'atlases', 'HarvardOxford', 'HarvardOxford-cort-maxprob-thr0-2mm.nii.gz')
sub_atlas_file = os.path.join(fsl_dir, 'data', 'atlases', 'HarvardOxford', 'HarvardOxford-sub-maxprob-thr0-2mm.nii.gz')

In [5]:
cort_atlas = nib.load(cort_atlas_file)
sub_atlas = nib.load(sub_atlas_file)

In [6]:
SPM_dir = os.path.join(out_dir, 'SPM', 'level3')
gain_dir = os.path.join(SPM_dir, 'gain')
loss_dir = os.path.join(SPM_dir, 'loss')

# Boolean image including thresholded activation > 0
print('SPM T thresh')
SPMT1_thresh = nib.load(os.path.join(gain_dir, 'spmT_0001_thresh.nii'))
SPMT1_thresh = math_img('np.nan_to_num(img1, 0)', img1=SPMT1_thresh)
print(sum(sum(sum(np.greater(SPMT1_thresh.get_data(), 0)))))


print('hypo 1 thresh')
hypo1_thresh = nib.load(os.path.join(gain_dir, 'hypo1_thresh.nii'))
hypo1_thresh = math_img('np.nan_to_num(img1, 0)', img1=hypo1_thresh)
print(sum(sum(sum(np.greater(hypo1_thresh.get_data(), 0)))))
print(np.amin(hypo1_thresh.get_data()[hypo1_thresh.get_data()!=0]))



gain_indif = math_img('np.greater(np.nan_to_num(img1, 0), 0)',
                      img1=nib.load(os.path.join(gain_dir, 'hypo1_thresh.nii')))
print(sum(sum(sum(gain_indif.get_data()))))


gain_range = math_img('np.greater(np.nan_to_num(img1, 0), 0)',
                      img1=nib.load(os.path.join(gain_dir, 'hypo2_thresh.nii')))

loss_indif_neg = math_img('np.greater(np.nan_to_num(img1, 0), 0)',
                          img1 = nib.load(os.path.join(loss_dir, 'hypo5_thresh.nii')))
loss_range_neg = math_img('np.greater(np.nan_to_num(img1, 0), 0)',
                          img1=nib.load(os.path.join(loss_dir, 'hypo6_thresh.nii')))
loss_indif = math_img('np.greater(np.nan_to_num(img1, 0), 0)',
                      img1=nib.load(os.path.join(loss_dir, 'hypo7_thresh.nii')))
loss_range = math_img('np.greater(np.nan_to_num(img1, 0), 0)',
                      img1=nib.load(os.path.join(loss_dir, 'hypo8_thresh.nii')))
loss_range_vs_indif = math_img('np.greater(np.nan_to_num(img1, 0), 0)',
                               img1=nib.load(os.path.join(loss_dir, 'hypo9_thresh.nii')))

SPM T thresh
64
hypo 1 thresh
64
4.337327
64


In [7]:
print(cort_atlas.header.get_data_shape())
print(sub_atlas.header.get_data_shape())
print(gain_indif.header.get_data_shape())

# Reslice atlas on SPM's MNI default resolution using nearest neighbours
cort_atlas_spm = resample_from_to(cort_atlas, gain_indif, order=0)
cort_atlas_spm.to_filename(os.path.join(res_dir, 'cort_atlas.nii.gz'))

sub_atlas_spm = resample_from_to(sub_atlas, gain_indif, order=0)
sub_atlas_spm.to_filename(os.path.join(res_dir, 'sub_atlas.nii.gz'))
print(cort_atlas_spm.header.get_data_shape())
print(sub_atlas_spm.header.get_data_shape())

(91, 109, 91)
(91, 109, 91)
(97, 115, 81)
(97, 115, 81)
(97, 115, 81)


In [8]:
VMPFC = math_img("np.multiply(atlas == 25, atlas)", atlas=cort_atlas_spm)
VMPFC.to_filename(os.path.join(res_dir, 'VMPFC.nii.gz'))
VStriatum = math_img("np.multiply(np.logical_or(atlas == 11, atlas == 21), atlas)", atlas=sub_atlas_spm)
VStriatum.to_filename(os.path.join(res_dir, 'VStriatum.nii.gz'))
Amygdala = math_img("np.multiply(np.logical_or(atlas == 10, atlas == 20), atlas)", atlas=sub_atlas_spm)
Amygdala.to_filename(os.path.join(res_dir, 'Amygdala.nii.gz'))

# Amygdala = np.logical_or(atlas_spm.get_data() == 10, atlas_spm.get_data() == 20)

In [9]:
import nilearn

In [10]:
# 1. Parametric effect of gain: Positive effect in ventromedial PFC - for the equal indifference group
print('1. Parametric effect of gain: Positive effect in ventromedial PFC - for the equal indifference group')
hyp1 = math_img("np.logical_and(np.greater(gain_indif, 0), np.greater(VMPFC, 0))", VMPFC=VMPFC, gain_indif=gain_indif)
print(sum(sum(sum(hyp1.get_data()))))
print(np.nonzero(hyp1.get_data()))
hyp1_img = math_img("np.multiply(hyp1, region)", hyp1=hyp1, region=VMPFC)
hyp1_img.to_filename(os.path.join(res_dir, 'hyp1_interest.nii.gz'))

# 2. Parametric effect of gain: Positive effect in ventromedial PFC - for the equal range group
print('2. Parametric effect of gain: Positive effect in ventromedial PFC - for the equal range group')
hyp2 = math_img("np.logical_and(VMPFC, gain_range > 0)", VMPFC=VMPFC, gain_range=gain_range)
print(sum(sum(sum(hyp2.get_data()))))

# 3. Parametric effect of gain: Positive effect in ventral striatum - for the equal indifference group
print('3. Parametric effect of gain: Positive effect in ventral striatum - for the equal indifference group')
hyp3 = math_img("np.logical_and(VStriatum, gain_indif > 0)", VStriatum=VStriatum, gain_indif=gain_indif)
print(sum(sum(sum(hyp3.get_data()))))

# 4. Parametric effect of gain: Positive effect in ventral striatum - for the equal range group
print('4. Parametric effect of gain: Positive effect in ventral striatum - for the equal range group')
hyp4 = math_img("np.logical_and(VStriatum, gain_range > 0)", VStriatum=VStriatum, gain_range=gain_range)
print(sum(sum(sum(hyp4.get_data()))))

# 5. Parametric effect of loss: Negative effect in VMPFC - for the equal indifference group
print('5. Parametric effect of loss: Negative effect in VMPFC - for the equal indifference group')
hyp5 = math_img("np.logical_and(VMPFC, loss_indif_neg > 0)", VMPFC=VMPFC, loss_indif_neg=loss_indif_neg)
print(sum(sum(sum(hyp5.get_data()))))

# 6. Parametric effect of loss: Negative effect in VMPFC - for the equal range group
print('6. Parametric effect of loss: Negative effect in VMPFC - for the equal range group')
hyp6 = math_img("np.logical_and(VMPFC, loss_range_neg > 0)", VMPFC=VMPFC, loss_range_neg=loss_range_neg)
print(sum(sum(sum(hyp6.get_data()))))

# 7. Parametric effect of loss: Positive effect in amygdala - for the equal indifference group
print('7. Parametric effect of loss: Positive effect in amygdala - for the equal indifference group')
hyp7 = math_img("np.logical_and(Amygdala, loss_indif > 0)", Amygdala=Amygdala, loss_indif=loss_indif)
print(sum(sum(sum(hyp7.get_data()))))

# 8. Parametric effect of loss: Positive effect in amygdala - for the equal range group
print('8. Parametric effect of loss: Positive effect in amygdala - for the equal range group')
hyp8 = math_img("np.logical_and(Amygdala, loss_range > 0)", Amygdala=Amygdala, loss_range=loss_range)
print(sum(sum(sum(hyp8.get_data()))))

# 9. Equal range vs. equal indifference: Greater response to losses in amygdala for equal range condition vs. equal indifference condition
print('9. Equal range vs. equal indifference: Greater response to losses in amygdala for equal range condition vs. equal indifference condition')
hyp9 = math_img("np.logical_and(Amygdala, loss_range_vs_indif > 0)", Amygdala=Amygdala, loss_range_vs_indif=loss_range_vs_indif)
print(sum(sum(sum(hyp9.get_data()))))

1. Parametric effect of gain: Positive effect in ventromedial PFC - for the equal indifference group
0
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
2. Parametric effect of gain: Positive effect in ventromedial PFC - for the equal range group
4
3. Parametric effect of gain: Positive effect in ventral striatum - for the equal indifference group
0
4. Parametric effect of gain: Positive effect in ventral striatum - for the equal range group
5
5. Parametric effect of loss: Negative effect in VMPFC - for the equal indifference group
318
6. Parametric effect of loss: Negative effect in VMPFC - for the equal range group
0
7. Parametric effect of loss: Positive effect in amygdala - for the equal indifference group
0
8. Parametric effect of loss: Positive effect in amygdala - for the equal range group
0
9. Equal range vs. equal indifference: Greater response to losses in amygdala for equal range condition vs. equal indifference condition
0


In [11]:
hypo1_thresh = nib.load(os.path.join(gain_dir, 'hypo1_thresh.nii'))

VMPFC_dat = nib.load(os.path.join(res_dir, 'VMPFC.nii.gz'))

aa = np.logical_and(np.greater(hypo1_thresh.get_data(), 0), np.greater(VMPFC_dat.get_data(), 0))
print(aa.any())

greater_0_hypo1_thresh = np.greater(hypo1_thresh.get_data(), 0)
print(sum(sum(sum(greater_0_hypo1_thresh))))
print(hypo1_thresh.get_data().shape)


False
64
(97, 115, 81)


/anaconda3/envs/narps/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in greater
  """


In [12]:
gain_dir

'/Volumes/cam/NARPS_out/SPM/level3/gain'

In [13]:
# 2. Parametric effect of gain: Positive effect in ventromedial PFC - for the equal range group
print('2. Parametric effect of gain: Positive effect in ventromedial PFC - for the equal range group')
hyp2 = math_img("np.logical_and(VMPFC, gain_range > 0)", VMPFC=VMPFC, gain_range=gain_range)
print(sum(sum(sum(hyp2.get_data()))))

# 3. Parametric effect of gain: Positive effect in ventral striatum - for the equal indifference group
print('3. Parametric effect of gain: Positive effect in ventral striatum - for the equal indifference group')
hyp3 = math_img("np.logical_and(VStriatum, gain_indif > 0)", VStriatum=VStriatum, gain_indif=gain_indif)
print(sum(sum(sum(hyp3.get_data()))))

# 4. Parametric effect of gain: Positive effect in ventral striatum - for the equal range group
print('4. Parametric effect of gain: Positive effect in ventral striatum - for the equal range group')
hyp4 = math_img("np.logical_and(VStriatum, gain_range > 0)", VStriatum=VStriatum, gain_range=gain_range)
print(sum(sum(sum(hyp4.get_data()))))

# 5. Parametric effect of loss: Negative effect in VMPFC - for the equal indifference group
print('5. Parametric effect of loss: Negative effect in VMPFC - for the equal indifference group')
hyp5 = math_img("np.logical_and(VMPFC, loss_indif_neg > 0)", VMPFC=VMPFC, loss_indif_neg=loss_indif_neg)
print(sum(sum(sum(hyp5.get_data()))))

# 6. Parametric effect of loss: Negative effect in VMPFC - for the equal range group
print('6. Parametric effect of loss: Negative effect in VMPFC - for the equal range group')
hyp6 = math_img("np.logical_and(VMPFC, loss_range_neg > 0)", VMPFC=VMPFC, loss_range_neg=loss_range_neg)
print(sum(sum(sum(hyp6.get_data()))))

# 7. Parametric effect of loss: Positive effect in amygdala - for the equal indifference group
print('7. Parametric effect of loss: Positive effect in amygdala - for the equal indifference group')
hyp7 = math_img("np.logical_and(Amygdala, loss_indif > 0)", Amygdala=Amygdala, loss_indif=loss_indif)
print(sum(sum(sum(hyp7.get_data()))))

# 8. Parametric effect of loss: Positive effect in amygdala - for the equal range group
print('8. Parametric effect of loss: Positive effect in amygdala - for the equal range group')
hyp8 = math_img("np.logical_and(Amygdala, loss_range > 0)", Amygdala=Amygdala, loss_range=loss_range)
print(sum(sum(sum(hyp8.get_data()))))

# 9. Equal range vs. equal indifference: Greater response to losses in amygdala for equal range condition vs. equal indifference condition
print('9. Equal range vs. equal indifference: Greater response to losses in amygdala for equal range condition vs. equal indifference condition')
hyp9 = math_img("np.logical_and(Amygdala, loss_range_vs_indif > 0)", Amygdala=Amygdala, loss_range_vs_indif=loss_range_vs_indif)
print(sum(sum(sum(hyp9.get_data()))))

2. Parametric effect of gain: Positive effect in ventromedial PFC - for the equal range group
4
3. Parametric effect of gain: Positive effect in ventral striatum - for the equal indifference group
0
4. Parametric effect of gain: Positive effect in ventral striatum - for the equal range group
5
5. Parametric effect of loss: Negative effect in VMPFC - for the equal indifference group
318
6. Parametric effect of loss: Negative effect in VMPFC - for the equal range group
0
7. Parametric effect of loss: Positive effect in amygdala - for the equal indifference group
0
8. Parametric effect of loss: Positive effect in amygdala - for the equal range group
0
9. Equal range vs. equal indifference: Greater response to losses in amygdala for equal range condition vs. equal indifference condition
0
